In [1]:
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "6_UJ-snFLq_FKHjweJhg"

CERT_FINGERPRINT = "1123dfb3b65fa37733335d06eddf4002585aee390b6e5047fc40e247ba723920"

es = Elasticsearch(
    "https://localhost:9200",
    ssl_assert_fingerprint=CERT_FINGERPRINT,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

if es.ping():
    print("connected!")

print(es.info()._body)

connected!
{'name': '7cbebeb542a3', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Dd9uvPVYTAaJV3SHr9HTgA', 'version': {'number': '9.1.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0c781091a2f57de895a73a1391ff8426c0153c8d', 'build_date': '2025-08-24T22:05:04.526302670Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [2]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [3]:
import json

dummy_data = json.load(open('dummy_data.json'))
dummy_data

document_ids = []

def insert_document(document):
    response = es.index(index='my_index', body=document)
    return response

for document in dummy_data:
    response = insert_document(document)
    document_ids.append(response['_id'])

document_ids

['J0py9ZgBDVTKBi1fo_WP', 'KEpy9ZgBDVTKBi1fo_Wr', 'KUpy9ZgBDVTKBi1fo_Wv']

# Update document

In [6]:
response = es.update(
    index='my_index',
    id=document_ids[0],
    script={
        'source': 'ctx._source.title = params.title',
        'params': {
            'title': 'new title 1'
        }
    }
)
response

ObjectApiResponse({'_index': 'my_index', '_id': 'J0py9ZgBDVTKBi1fo_WP', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1})

In [8]:
response = es.get(index='my_index', id=document_ids[0])
print(response.body)

{'_index': 'my_index', '_id': 'J0py9ZgBDVTKBi1fo_WP', '_version': 3, '_seq_no': 4, '_primary_term': 1, 'found': True, '_source': {'title': 'new title 1', 'text': 'text 1', 'created_on': '2024-08-27'}}


# Add new field

To add a new field, you can use the script argument or the doc argument

In [10]:
response = es.update(
    index="my_index",
    id=document_ids[0],
    script={
        "source": "ctx._source.new_field = 'dummy_value'",
    },
)
print(response.body)

{'_index': 'my_index', '_id': 'J0py9ZgBDVTKBi1fo_WP', '_version': 5, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 1}


In [11]:
response = es.update(
    index="my_index",
    id=document_ids[0],
    doc={
        "new_value_2": "dummy_value_2",
    },
)
print(response.body)

{'_index': 'my_index', '_id': 'J0py9ZgBDVTKBi1fo_WP', '_version': 6, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7, '_primary_term': 1}


# Remove a field

In [12]:
response = es.update(
    index="my_index",
    id=document_ids[0],
    script={
        "source": "ctx._source.remove('new_field')",
    },
)
print(response.body)

{'_index': 'my_index', '_id': 'J0py9ZgBDVTKBi1fo_WP', '_version': 7, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 8, '_primary_term': 1}


# If documents doesn't exist in the index

In [13]:
response = es.update(
    index="my_index",
    id="1",
    doc={
        "book_id": 1234,
        "book_name": "A book",
    },
    doc_as_upsert=True,
)

response

ObjectApiResponse({'_index': 'my_index', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 9, '_primary_term': 1})